### Programming for Biomedical Informatics
#### Week 9 - Functional Analysis Using Ontologies

Ontologies are commonly used to aid interpretation of molecular data, most commonly through use of functional annotations to genes and proteins using the Gene Onotlogy combined with downstream likelihood/enrichment analysis using tools such as GSEA as we have discussed. Ontologies are also used in strategies to align unstructured data with domains, for example looking for words and/or phrases that can be mapped to classes in ontologies. Examples here would include things like looking for terms associated with clinical terminolgies in patient discharge summaries.

In this notebook we will perform some basic phenotype extraction from publication abstracts, attempting to find examples of HPO terms that are associated with mentions of particular diseases.

To do this we will randomly select 1000 papers from PubMed that are tagged with the MeSH Major Topic "Autism Spectrum Disorder" retreive their titles and abstracts and then search for phenotypes and genes.

In [ ]:
'''
we're going to use a biomedical named entity recognition model called en_core_sci_sm which is a model developed by the Allen Institute for biomedical text processing
https://allenai.github.io/scispacy/
'''

# %pip install scispacy
# %pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.4/en_core_sci_sm-0.5.4.tar.gz

In [5]:
# Setup for NLP

import pandas as pd
import spacy
from scispacy.linking import EntityLinker
from scispacy.abbreviation import AbbreviationDetector

# supress warnings
import warnings
warnings.filterwarnings('ignore')

In [6]:
'''when this is first run it will download some large data files to perform the UMLS linking
approx 2GB in total. On subsequent rune the model will take about 1 minute to load'''

# load the model
nlp = spacy.load("en_core_sci_sm");

# add abbreviations detector
nlp.add_pipe("abbreviation_detector");

# add UMLS entity-linker
nlp.add_pipe("scispacy_linker", config={"resolve_abbreviations": True,"linker_name": "umls","filter_for_definitions": False});

In [7]:
# let's look at a simple example
queryText = 'PAX6 and GLI3 is a highly conserved transcription factor that plays a critical role in eye development in all animals. Mutations in the PAX6 gene are associated with aniridia, a congenital eye malformation characterized by the absence of the iris and other eye abnormalities.'

concepts = dict()

try:
    #perform nlp
    doc = nlp(queryText)
    for entity in doc.ents:
        link = concept,score = entity._.kb_ents[0]
        concepts[entity.text] = concept
        print(entity.text, concept, score)
except:
    #case of no text
    pass

PAX6 C1418276 0.9495171904563904
GLI3 C0082708 0.9556154608726501
eye development C1517080 0.9721702337265015
animals C0003062 0.973741888999939
Mutations C0026882 0.9732257127761841
PAX6 gene C1418276 0.8935739994049072
associated with C0332281 0.9891292452812195
aniridia C0003076 0.9635230898857117
congenital eye malformation C0015393 0.9502193927764893
characterized C1880022 0.9680079817771912
absence C0332197 0.9857189655303955
iris C0022077 0.9765904545783997
eye abnormalities C0015393 0.9852664470672607


In [8]:
# the human phenotype ontology contains database_cross_reference entries that include the UMLS concept id
# we can use this to link the HPO terms to the UMLS concepts

# load the HPO data using pronto
import pronto

# load the HPO ontology
# fetch the Human Phenotype Onology OBO file and parse it with pronto

# download the HPO ontology OBO file
import urllib.request

current_hpo_url = 'http://purl.obolibrary.org/obo/hp.obo'

# download the file
urllib.request.urlretrieve(current_hpo_url,'hpo.obo');

# parse the file
hpo = pronto.Ontology('hpo.obo')


In [9]:
# we can look in the xrefs (sic. cross-references) of a term to find the UMLS concept id
def hpo2concept(hpo_id):
    term = hpo[hpo_id]
    xrefs = [xref.id for xref in term.xrefs]
    try:
        umls_id = [xref for xref in xrefs if xref.startswith('UMLS')][0].split(':')[1]
        return umls_id
    except:
        return None

# let's test this function
hpo2concept('HP:0001695')

'C0018790'

In [10]:
# we're now going to brute force the conversion of all HPO terms to UMLS concepts
hpo2umls = {term.id:hpo2concept(term.id) for term in hpo.terms()}

In [11]:
# look at the first 10 entries
list(hpo2umls.items())[:10]

[('HP:0000001', 'C0444868'),
 ('HP:0000002', 'C4025901'),
 ('HP:0000003', 'C3714581'),
 ('HP:0000005', 'C1708511'),
 ('HP:0000006', 'C0443147'),
 ('HP:0000007', 'C0441748'),
 ('HP:0000008', 'C4025900'),
 ('HP:0000009', 'C3806583'),
 ('HP:0000010', 'C0262655'),
 ('HP:0000011', 'C0005697')]

In [12]:
#lets see if any of the CUIs from our test sentence have been mapped to HPO terms
for entity in concepts.keys():
    concept = concepts[entity]
    if concept in hpo2umls.values():
        print(entity, concept, [hpo[k] for k,v in hpo2umls.items() if v == concept])

aniridia C0003076 [Term('HP:0000526', name='Aniridia')]
congenital eye malformation C0015393 [Term('HP:0000478', name='Abnormality of the eye')]
eye abnormalities C0015393 [Term('HP:0000478', name='Abnormality of the eye')]


In [26]:
# lets 

current_asdpto_url = 'https://data.bioontology.org/ontologies/ASDPTO/submissions/1/download?apikey=4a2fbff0-ef88-432e-b1a1-dffc07e71146'

# download the file
urllib.request.urlretrieve(current_asdpto_url,'autism.obo');

# parse the file
autism  = pronto.Ontology('autism.obo')


In [ ]:
'''
As a niche example (not a mainstream ontology) we will look at the ASDPTO ontology which
has been developed as a custom ontology for autism spectrum disorder.

For every term in the ASDPTO ontology, we will look for UMLS concepts.

NB we are limited to the work done by ASDPTO curators in adding annotations to the terms
'''

# function to find the UMLS concept for a term in ASDPTO
def find_concept(term):
    for annotation in term.annotations:
        try:
            # if the string contains a cui= then it is a UMLS concept
            # extract the CUI
            if 'cui=' in annotation.resource:
                #split the string on 'cui=' and take the remainder
                concept = annotation.resource.split('cui=')[1]
                return(concept)
        except:
            pass

# we can now use this function to find the UMLS concept for each term in the ASDPTO ontology
asdpto2umls = {term.name:find_concept(term) for term in autism.terms()}

# remove any None entries
asdpto2umls = {k:v for k,v in asdpto2umls.items() if v is not None}

# print how many terms have been mapped to UMLS concepts
print(f'There are ',len(asdpto2umls),' terms in the ASDPTO ontology that have been mapped to UMLS concepts')

# look at the first 10 entries
list(asdpto2umls.items())[:10]

There are  82  terms in the ASDPTO ontology that have been mapped to UMLS concepts


[('Psychiatric Hospitalization', 'C0748061'),
 ('Ability to Convey Feelings', 'C1821397'),
 ('Gross Motor Skills', 'C0678858'),
 ('Perinatal Exposures', 'C1531967'),
 ('Use of Free Time', 'C0517871'),
 ('Depression', 'C0011570'),
 ('Medical History', 'C0262926'),
 ('Social Anxiety', 'C0424166'),
 ('Newborn Exposures', 'C0920206'),
 ('Adherence to Rules in the Home', 'C0562410')]

In [ ]:
'''Now that we have all the NLP components in place to identify and map HPO terms let's now fetch the data and perform the analysis'''

In [100]:
# let's use our knowledge of eUtils to fetch the raw material for our analysis
# we will use the requests library to fetch the data using the eUtils API
# we will use the xml library to parse the data

import urllib.request
import xml.etree.ElementTree as ET

# load my API key from the file
with open('../api_keys/ncbi.txt', 'r') as file:
    api_key = file.read().strip()

with open('../api_keys/ncbi_email.txt', 'r') as file:
    email = file.read().strip()

pubmed_query = '"Autism Spectrum Disorder[Majr]"'

# Define the parameters for the eSearch request
esearch_params = {
    'db': 'pubmed',
    'term': pubmed_query,
    'api_key': api_key,
    'email': email,
    'usehistory': 'y'
}

# encode the parameters so they can be passed to the API
encoded_data = urllib.parse.urlencode(esearch_params).encode('utf-8')

# the base request url for eSearch
url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"

# make the request
request = urllib.request.Request(url, data=encoded_data)
response = urllib.request.urlopen(request)

# read into an XML object
esaerch_data_XML = ET.fromstring(response.read())

# print the number of results
count = esaerch_data_XML.find('Count').text
print(f'Total number of results: {count}')

# Extract WebEnv and QueryKey
webenv = esaerch_data_XML.find('WebEnv').text
query_key = esaerch_data_XML.find('QueryKey').text

efetch_params = {
'db': 'pubmed',
'query_key': query_key,
'WebEnv': webenv,
'retmax': '1000',
'api_key': api_key,
'email': email
}

# encode the parameters so they can be passed to the API
encoded_data = urllib.parse.urlencode(efetch_params).encode('utf-8')

# the base request url for eSearch
url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi"

# make the request
request = urllib.request.Request(url, data=encoded_data)
response = urllib.request.urlopen(request)

# read into an XML object
efetch_data_XML = ET.fromstring(response.read())

# let's look at the first 10 articles
for article in efetch_data_XML.findall('.//PubmedArticle')[:10]:
    pmid = article.find('.//PMID').text
    title = article.find('.//ArticleTitle').text
    abstract = article.find('.//AbstractText').text
    print(f'{pmid}: {title}')
    print(f'Abstract: {abstract}')

Total number of results: 42005
39537371: Language defi cit in boys with autism spectrum disorder in relation to maternal reproductive health, endocrine disruptors, and delivery method.
Abstract: The presented research aimed to identify prenatal factors involved in abnormal neurodevelopment and postnatal manifestation of an autistic phenotype in 107 boys (average age 4.31 ± 2.24 years).
39536207: Parental emotions in families of children and adolescents with and without autism spectrum disorder.
Abstract: This study assessed the influence of sociodemographic conditions, oral hygiene habits, and the socio-psychological need for orthodontic treatment on the emotions of caregivers of children with and without autism spectrum disorder (ASD). We conducted a comparative cross-sectional study with families of individuals aged 6 to 14 years at a reference center for neurodevelopmental disorders and two dental school clinics in northeastern Brazil. Caregiver emotions were assessed using the 'Par

In [101]:
# for each article check whether it has an abstract and a title
# if it does combine the title and abstract into a single string
# if it doesn't remove it from the list
articles = dict()

for article in efetch_data_XML.findall('.//PubmedArticle'):
    try:
        pmid = article.find('.//PMID')
        title = article.find('.//ArticleTitle')
        abstract = article.find('.//AbstractText')
        tiab = title.text + ' ' + abstract.text
        articles[pmid.text] = tiab
    except:
        pass

print(f'Number of articles with abstracts: {len(articles)}')

Number of articles with abstracts: 931


In [104]:
# lets write a function based on code above to perform ner on articles and return
def nlp_article(article):

    current_article_concepts = dict()

    try:
        #perform nlp
        doc = nlp(article)
        for entity in doc.ents:
            link = concept,score = entity._.kb_ents[0]
            current_article_concepts[entity.text] = concept
    except:
        #case of no text
        pass

    hpo_terms = []

    for entity in current_article_concepts.keys():
        concept = current_article_concepts[entity]
        if concept in hpo2umls.values():
            print(entity, concept, [hpo[k] for k,v in hpo2umls.items() if v == concept])
            current_hpo = [hpo[k] for k,v in hpo2umls.items() if v == concept]
            hpo_terms.append(current_hpo[0].id)
    return list(set(hpo_terms))

In [105]:
# now we can apply this function to all the articles
articles_hpo = {pmid: nlp_article(article) for pmid, article in articles.items()}

autistic C0004352 [Term('HP:0000717', name='Autism')]
Autism C0004352 [Term('HP:0000717', name='Autism')]
autistic C0004352 [Term('HP:0000717', name='Autism')]
autism C0004352 [Term('HP:0000717', name='Autism')]
Autism C0004352 [Term('HP:0000717', name='Autism')]
Cerebral Palsy C0007789 [Term('HP:0100021', name='Cerebral palsy')]
autism C0004352 [Term('HP:0000717', name='Autism')]
Autistic People C0004352 [Term('HP:0000717', name='Autism')]
Anxiety C0003467 [Term('HP:0000739', name='Anxiety')]
anxiety C0003467 [Term('HP:0000739', name='Anxiety')]
autism C0004352 [Term('HP:0000717', name='Autism')]
pain insensitivity C0344307 [Term('HP:0007021', name='Pain insensitivity')]
PAI C0344307 [Term('HP:0007021', name='Pain insensitivity')]
Autistic C0004352 [Term('HP:0000717', name='Autism')]
anxiety C0003467 [Term('HP:0000739', name='Anxiety')]
autism C0004352 [Term('HP:0000717', name='Autism')]
moderate C0205081 [Term('HP:0012826', name='Moderate')]
autistic people C0004352 [Term('HP:0000717

In [18]:
# let's look at the results from the first 10 articles
list(articles_hpo.items())[:10]

# what percentage of articles have HPO terms
articles_with_hpo = [k for k,v in articles_hpo.items() if v]
print(f'Percentage of articles with HPO terms: {len(articles_with_hpo)/len(articles)*100:.2f}%')

Percentage of articles with HPO terms: 37.59%


In [19]:
# find the unique HPO terms found in the articles
unique_hpo_terms = list(set([term for terms in articles_hpo.values() for term in terms]))

# create a dataframe to store the data
df = pd.DataFrame(index=articles.keys(), columns=unique_hpo_terms)

# fill the dataframe
for pmid, terms in articles_hpo.items():
    df.loc[pmid, terms] = 1

# fill the NaN values with 0
df.fillna(0, inplace=True)

# print the first 5 rows
df.head()

,HP:0000733,HP:0000365,HP:0012837,HP:0005978,HP:0000738,HP:0012531,HP:0002878,HP:0030858,HP:0001370,HP:0002725,...,HP:0012768,HP:0002373,HP:0000271,HP:0012838,HP:0100716,HP:0011009,HP:0000750,HP:0001626,HP:0007021,HP:0002037
39537371,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
39536207,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
39533234,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
39514479,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
39510775,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
# count the number of times each term appears and store this in a dataframe with columns
# 'HPO Term Name', 'HPO Term', 'Count' and sort by count
hpo_counts = df.sum().sort_values(ascending=False).reset_index()
hpo_counts.columns = ['HPO Term', 'Count']
hpo_counts['HPO Term Name'] = [hpo[term].name for term in hpo_counts['HPO Term']]

# use PrettyTable to display the data
from prettytable import PrettyTable

table = PrettyTable()
table.field_names = hpo_counts.columns
for row in hpo_counts.itertuples(index=False):
    table.add_row(row)
print(table)

+------------+-------+---------------------------------------------------------------+
|  HPO Term  | Count |                         HPO Term Name                         |
+------------+-------+---------------------------------------------------------------+
| HP:0000717 |  240  |                             Autism                            |
| HP:0007018 |   24  |            Attention deficit hyperactivity disorder           |
| HP:0000739 |   15  |                            Anxiety                            |
| HP:0001631 |   13  |                      Atrial septal defect                     |
| HP:0012828 |   10  |                             Severe                            |
| HP:0000733 |   7   |                        Motor stereotypy                       |
| HP:0001250 |   7   |                            Seizure                            |
| HP:0000729 |   6   |                       Autistic behavior                       |
| HP:0001513 |   6   |                     